In [86]:
import pandas as pd
import os
import sys
from typing import Text
 
from absl import logging
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

In [67]:
df = pd.read_csv('./data//fake_or_real_news.csv')
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [68]:
df.rename(columns={'Unnamed: 0': 'id', 'label': 'is_fake'}, inplace=True)
df.head()

,id,title,text,is_fake
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [69]:
df.is_fake = df.is_fake.map({'FAKE': 1, 'REAL': 0})
df.is_fake.value_counts()

0    3171
1    3164
Name: is_fake, dtype: int64

In [76]:
df.to_csv('./data//fake_or_real_news.csv', index=False)
df.head()

,id,title,text,is_fake
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",1
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,0
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",1
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,0


In [77]:
df.isnull().sum()

id         0
title      0
text       0
is_fake    0
dtype: int64

In [84]:
PIPELINE_NAME = "shendyeff-fake-news-pipeline"

# pipeline inputs
DATA_ROOT = "data"
TRANSFORM_MODULE_FILE = "modules/fake_news_transform.py"
TRAINER_MODULE_FILE = "modules/fake_news_trainer.py"
TUNER_MODULE_FILE = "modules/fake_news_tuner.py"

# pipeline outputs
OUTPUT_BASE = "output"
serving_model_dir = os.path.join(OUTPUT_BASE, 'serving_model')
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

In [87]:
def init_local_pipeline(
    components, pipeline_root: Text
) -> pipeline.Pipeline:
    
    logging.info(f"Pipeline root set to: {pipeline_root}")
    beam_args = [
        "--direct_running_mode=multi_processing"
        # 0 auto-detect based on on the number of CPUs available 
        # during execution time.
        "----direct_num_workers=0" 
    ]
    
    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path
        ),
        eam_pipeline_args=beam_args
    )

In [88]:
if __name__ == "__main__":
    logging.set_verbosity(logging.INFO)
    
    from modules.components import init_components
    
    components = init_components(
        DATA_ROOT,
        training_module=TRAINER_MODULE_FILE,
        transform_module=TRANSFORM_MODULE_FILE,
        tuner_module=TUNER_MODULE_FILE,
        training_steps=5000,
        eval_steps=1000,
        serving_model_dir=serving_model_dir,
    )
    
    pipeline = init_local_pipeline(components, pipeline_root)
    BeamDagRunner().run(pipeline=pipeline)

INFO:absl:Finished tuning... Tuner ID: tuner0
INFO:absl:Best HyperParameters: {'space': [{'class_name': 'Choice', 'config': {'name': 'embedding_dim', 'default': 32, 'conditions': [], 'values': [32, 64], 'ordered': True}}, {'class_name': 'Choice', 'config': {'name': 'lstm_units', 'default': 16, 'conditions': [], 'values': [16, 32], 'ordered': True}}, {'class_name': 'Int', 'config': {'name': 'dense_units', 'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': 'linear'}}, {'class_name': 'Float', 'config': {'name': 'dropout_rate', 'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.3, 'step': 0.1, 'sampling': 'linear'}}, {'class_name': 'Choice', 'config': {'name': 'learning_rate', 'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001], 'ordered': True}}, {'class_name': 'Int', 'config': {'name': 'num_layers', 'default': None, 'conditions': [], 'min_value': 2, 'max_value': 4, 'step': 1, 'sampling': 'linear'}}], 'values': {'embeddi

Trial 1 Complete [00h 00m 34s]
val_binary_accuracy: 0.8521126508712769

Best val_binary_accuracy So Far: 0.8521126508712769
Total elapsed time: 00h 00m 34s
Results summary
Results in output\shendyeff-fake-news-pipeline\Tuner\.system\executor_execution\8\.temp\8\email_spam_tuner
Showing 10 best trials
Objective(name="val_binary_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
embedding_dim: 32
lstm_units: 16
dense_units: 128
dropout_rate: 0.1
learning_rate: 0.001
num_layers: 3
Score: 0.8521126508712769


INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.evaluator.component.Evaluator"
    base_type: EVALUATE
  }
  id: "Evaluator"
}
contexts {
  contexts {
    type {
      name: "pipeline"
    }
    name {
      field_value {
        string_value: "shendyeff-fake-news-pipeline"
      }
    }
  }
  contexts {
    type {
      name: "pipeline_run"
    }
    name {
      field_value {
        string_value: "20250212-002102.390232"
      }
    }
  }
  contexts {
    type {
      name: "node"
    }
    name {
      field_value {
        string_value: "shendyeff-fake-news-pipeline.Evaluator"
      }
    }
  }
}
inputs {
  inputs {
    key: "baseline_model"
    value {
      channels {
        producer_node_query {
          id: "Latest_blessed_model_resolver"
        }
        context_queries {
          type {
            name: "pipeline"
          }
          name {
            field_value {
              string_value: "shendyeff-fake-news-pipeline"
            }


In [75]:
# import tensorflow_model_analysis as tfma
# import json

# # Path ke file .tfrecord hasil evaluasi
# eval_result_path = "output/shendyeff-fake-news-pipeline/Evaluator/evaluation/9/metrics-00000-of-00001.tfrecord"

# # Memuat hasil evaluasi dari file .tfrecord
# eval_result = tfma.load_eval_result(eval_result_path)

# # Memeriksa struktur eval_result
# print(eval_result)

# # Menyusun data metrik dari eval_result
# eval_metrics = eval_result['metrics'] if 'metrics' in eval_result else {}

# # Konversi hasil evaluasi menjadi format JSON
# eval_result_json = {
#     "metrics": eval_metrics
# }

# # Menyimpan hasil evaluasi dalam format JSON
# with open("eval_result.json", "w") as f:
#     json.dump(eval_result_json, f, indent=4)

# print("Hasil evaluasi telah disimpan dalam format JSON.")
